In [68]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import joblib


In [69]:
def load_and_preprocess_data(file_path):
    """
    Load the dataset, rename relevant columns, and select only the necessary columns.
    """
    # Load dataset
    data = pd.read_csv(file_path)
    
    # Rename columns for clarity
    data = data.rename(columns={
        'startingAirport': 'origin_airport', 
        'destinationAirport': 'destination_airport',
        'totalFare': 'fare'
    })
    
    
    # Extract cabin type
    data['cabin_type'] = data['segmentsCabinCode'].apply(lambda x: x.split('||')[0])
    
    # Select and return only the relevant columns
    data = data[['origin_airport', 'destination_airport','segmentsDepartureTimeRaw', 'cabin_type', 'fare']]
    return data


In [70]:
# Function to extract first text before first delimiter, convert to datetime, round to nearest 30 mins, and convert to epoch seconds
def convert_to_epoch_seconds(time_str):
    try:
        # Extract the first time segment and convert to datetime
        time = pd.to_datetime(time_str.split('||')[0], utc=True, errors='coerce')
        # Round to the nearest 30 minutes
        time = (time.floor('30T') + pd.Timedelta(minutes=15)).floor('30T')
        # Convert to epoch seconds
        return int(time.timestamp())
    except Exception as e:
        # Handle any errors and return None or a default value
        return None

# Encode categorical columns and apply convert_to_epoch_seconds function
def encode_data(data):
    """
    Encode categorical columns in the dataset using Label Encoding.
    Apply convert_to_epoch_seconds function to 'segmentsDepartureTimeRaw' column.
    """
    # Initialize LabelEncoders
    label_encoders = {
        'origin_airport': LabelEncoder(),
        'destination_airport': LabelEncoder(),
        'cabin_type': LabelEncoder()
    }
    
    # Apply encoding and save encoders for each column
    for column, encoder in label_encoders.items():
        data[column] = encoder.fit_transform(data[column])
    
    # Apply convert_to_epoch_seconds function to 'segmentsDepartureTimeRaw' column
    data['departure_time_seconds'] = data['segmentsDepartureTimeRaw'].apply(convert_to_epoch_seconds)
    
    return data, label_encoders

In [72]:
# Train model
def train_model(X_train, y_train):
    """
    Train a  Adaboost model with specified parameters.
    """
    # Initialize Random Forest with parameters for reduced model size
    model  = AdaBoostRegressor(random_state=0, n_estimators=100)
    
    # Train the model
    model.fit(X_train, y_train)
    
    return model

In [76]:
# Split the data
def split_data(data, test_size=0.2, sample_fraction=0.1, random_state=42):
    """
    Sample a fraction of the data, separate features and target, and split into train/test sets.
    """
    # Take a random sample for quicker training
    sampled_data = data.sample(frac=sample_fraction, random_state=random_state)
    
    # Define features and target
    X = sampled_data[['origin_airport', 'destination_airport','departure_time_seconds', 'cabin_type']]
    y = sampled_data['fare']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test

In [73]:
# Evaluate model
def evaluate_model(model, X_test, y_test):
    """
    Predict and evaluate the model on test data.
    """
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print("Mean Absolute Error:", mae)
    print("R² Score:", r2)
    print("RMSE:", rmse)
    
    return mae, r2, rmse

In [74]:
# Define main pipeline function
def main_pipeline(file_path, model_path, encoders_path_prefix):
    """
    Main pipeline for loading data, preprocessing, training, and saving the model.
    """
    # Load and preprocess data
    data = load_and_preprocess_data(file_path)
    
    # Encode categorical data
    data, label_encoders = encode_data(data)
    
    # Split data
    X_train, X_test, y_train, y_test = split_data(data)
    
    # Train model
    model = train_model(X_train, y_train)
    
    # Evaluate model
    evaluate_model(model, X_test, y_test)
    
    # Save model and encoders
    #save_model_and_encoders(model, label_encoders, model_path, encoders_path_prefix)

In [77]:
# Execute the pipeline
if __name__ == "__main__":
    file_path = '/Users/thusondube/Downloads/sample_flights.csv'
    model_path = '/Users/thusondube/Projects/at3_mla/at3-model-experimentation/models/rf_model.pkl'
    encoders_path_prefix = '/Users/thusondube/Projects/at3_mla/at3-model-experimentation/model'

    main_pipeline(file_path, model_path, encoders_path_prefix)


/var/folders/p1/f_nj01q561bd83ht98rlq03r0000gn/T/ipykernel_26584/2668009095.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time = (time.floor('30T') + pd.Timedelta(minutes=15)).floor('30T')


Mean Absolute Error: 199.2348214841979
R² Score: -0.1455158358513191
RMSE: 259.6437666892042


In [80]:
X_train, X_test, y_train, y_test = split_data(data)

In [81]:
from code_pkg.models.performance import fit_assess_regressor

model1 = fit_assess_regressor(AdaBoostRegressor(random_state=0, n_estimators=100, learning_rate=0.05), X_train, y_train, X_test, y_test)



RMSE Training: 231.1976592233844
MAE Training: 171.9465263516327
RMSE Validation: 228.63143228968687
MAE Validation: 170.04998039702687


In [84]:
from joblib import dump

dump(model1, '/Users/thusondube/Documents/UTS2024/Projects/thuso_dube_A3_24542250_ada_backend/models/ada_reg1.joblib')



['/Users/thusondube/Documents/UTS2024/Projects/thuso_dube_A3_24542250_ada_backend/models/ada_reg1.joblib']